In [1]:
import sys
sys.path.append('/home/alexabades/recsys')
import pandas as pd 
import numpy as np 
import os

In [2]:
from src.data.Contextual import PreProcessDataNCFContextual

frappe_raw_path = '../data/raw/frappe/'
data = PreProcessDataNCFContextual(frappe_raw_path)

In [3]:
data.save_data(folder_name='frappe2')

Saved in:  /home/alexabades/recsys/src/data/processed/frappe2/frappe2


In [8]:
data2 = PreProcessDataNCFContextual(frappe_raw_path, min_interactions=2)

In [9]:
data2.save_data(folder_name='frappe5')

Saved in:  /home/alexabades/recsys/src/data/processed/frappe5/frappe5


In [15]:
frappe_raw_path = '../data/raw/frappe/frappe.csv'
frappe_processed_path = '../data/processed/frappeCtxA/frappeCtxA.train.rating'
df_raw = pd.read_csv(frappe_raw_path, sep='\t')
df_pro = pd.read_csv(frappe_processed_path, sep='\t', header=None)

In [16]:
num_users_raw = len(df_raw['user'].unique())
num_items_raw = len(df_raw['item'].unique())

In [25]:
num_users_pro = len(df_pro[0].unique())
num_items_pro = len(df_pro[1].unique())
num_users_pro,num_items_pro 

(651, 1127)

In [30]:
(num_users_raw - num_users_pro)/num_users_raw * 100
(num_items_raw - num_items_pro) / num_items_raw * 100

72.39098481136698

In [62]:
df_raw.shape[0], df_pro.shape[0]

(96203, 83722)

In [57]:
elements = df_raw[df_raw['user'] == 1].sample(1)

elements.index

Index([207], dtype='int64')

In [58]:
idx = []
idx.extend(elements.index)


In [59]:
idx

[207]

In [43]:
grouped = df_raw.groupby('user')['item'].count()
grouped.head()

user
0     25
1    360
2    363
3    166
4    920
Name: item, dtype: int64

In [44]:
grouped[0]

25

In [ ]:
    def _initialize_train_test(self, data: DataFrame, min_samples_test_set: int):
        """
        Funtion that splits the dataser into train/test.
        Following the strategy of Leave one out - test set 1 interaction per user


        TODO: Update, we have the prefilter.
        """
        frequency_interaction = data.groupby(self.user_column)[
            [self.item_column, self.ratings_column]
        ].count()
        users_one_interaction = frequency_interaction[
            frequency_interaction[self.ratings_column] == 1
        ].index
        list_of_users = data[self.user_column].unique()
        users_more_one_interaction = list(
            set(list_of_users) - set(users_one_interaction)
        )
        idx = []
        for i in users_more_one_interaction:
            element = data[data["user"] == i].sample(n=min_samples_test_set)
            idx.append(element.index[0])

        test_idx = np.isin(data.index, np.array(idx))

        train_ratings, test_ratings = (
            data[~test_idx],
            data[test_idx],
        )
        return train_ratings, test_ratings